# Preliminaries


In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [2]:
seed = 1602

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [3]:
#CALLBACKS FUNCTION ADAPTED TO THIS NEW TASK
from datetime import datetime

#WE EXTEND THE CONCEPT OF CALLBACK BY COMBINING TWO AND OTHER STUFF WE NEED
def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S') #DATE GENERATION

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now)) #WE ADD THE DATE TO THE NAME SO THAT WE CAN MONITOR AND COMAPRE RESULTS
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  #THIS FUNCTION ALLOWS TO SAVE THE MODEL DURING TRAINING
  #TAKES AS ARGUMENT WHERE WE WANT TO SAVE
  #BETTER TO SAVE ENTIRE MODEL AND NOT ONLY THE WEIGHTS
  #PUT SAVEBESTONLY TO FALSE, IT WILL SAVE THE LAST ONE
  #YOU CAN SAVE BEST BY USING EARLYSTOPPING! DONE BELOW
  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  #THIS WILL SAVE INFO ON THE METRICS ETC.
  #LOG DIR IS WHERE WE SAVE THE INFO
  #PROFILE BATCH TO ZERO HELPS REDUCE TIME
  #HISTOGRAM FREQ TELLS ON HOW MANY EPOCHS YOU NEED TO SAVE RESULTS
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)
  rop_callback =  tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
  callbacks.append(rop_callback)
  return callbacks

# Data Loading 



In [4]:
dataset = pd.read_csv('../input/dataset/dataset.csv')
print(dataset.shape)
dataset.head()
#WE LOAD DATA AND HAVE A LOOK AT SOME SAMPLES

In [5]:
dataset.info() 
#VARIABLES INFO

# Data Visualization


In [6]:
#A FUNCTION TO PLOT THE SERIES
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()

In [7]:
inspect_dataframe(dataset, dataset.columns) #WE VISUALIZE THE TRENDS IN THE VARIABLES 

# Splitting


In [8]:
#THE LAST SAMPLES OF THE GIVEN SERIES ARE USED FOR TESTING,
#THE NUMBER IS CHOSEN USING SAME PROPORTION USED BY LATTARI (CIRCA 9%)
test_size = 6500
X_train_raw = dataset.iloc[:-test_size]
# y_train_raw = y.iloc[:-test_size]
X_test_raw = dataset.iloc[-test_size:]
# y_test_raw = y.iloc[-test_size:]
print(X_train_raw.shape, X_test_raw.shape)

# Normalize both features and labels
X_min = X_train_raw.min()
X_max = X_train_raw.max()

#THEN WE NORMALIZE IN 0 1 RANGE
X_train_raw = (X_train_raw-X_min)/(X_max-X_min)
X_test_raw = (X_test_raw-X_min)/(X_max-X_min)

plt.figure(figsize=(17,5))
plt.plot(X_train_raw.Sponginess, label='Train (Sponginess)')
plt.plot(X_test_raw.Sponginess, label='Test (Sponginess)')
plt.title('Train-Test Split')
plt.legend()
plt.show()

plt.figure(figsize=(17,5))
plt.plot(X_train_raw.Crunchiness, label='Train (Crunchiness)')
plt.plot(X_test_raw.Crunchiness, label='Test (Crunchiness)')
plt.title('Train-Test Split')
plt.legend()
plt.show()


#Sequences (Direct Forecasting)

In [9]:
window = 2400
stride = 10

In [10]:
#THIS WILL BE USED AT END OF THE NOTEBOOK AND IS THE VERY LAST SEQUENCE AT THE END OF THE DATASET
#WILL BE USED AS STARTING POINT TO PREDICT THE FUTURE

future = dataset[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0)
future.shape

In [11]:
def build_sequences(df, target_labels=['Sponginess'], window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [12]:
target_labels = dataset.columns #WE TAKE ALL COLUMNS AS LABELS
telescope = 864

In [13]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_test, y_test = build_sequences(X_test_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

#WE CAN SEE DIMENSION OF THE CUTS AND THE DIMENSIONS OF WHAT WE WANT TO PREDICT 

In [14]:
#WE CAN SEE SOME EXAMPLES OF WINDOWS(?) 
def inspect_multivariate(X, y, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

In [15]:
inspect_multivariate(X_train, y_train, target_labels, telescope)

# Sequences (AutoReg)


In [16]:
target_labels = dataset.columns
window = 432
stride = 12
telescope = 144

#WE WANT TO LEARN ONLY ONE VALUE AT A TIME
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_test, y_test = build_sequences(X_test_raw, target_labels, window, stride, telescope)
#WE PRINT THE SHAPES OF OUR TRAIN AND TEST SEQUENCES
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [17]:
inspect_multivariate(X_train, y_train, target_labels, telescope)
#THE YELLOW POINT IS WHAT WE WANT TO PREDICT

In [18]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 100
print("output shape is", output_shape)
print("input shape is", input_shape)

# Hyperparameter Research

In [19]:
import keras_tuner as kt

In [20]:
def build_CONV_LSTM_model_for_random_search(hp):
    # Build the neural network layer by layer
    #with this instrument we have to manually define input size and output size because only arg needs to be the one for
    #the choices in the search, hp
    input_shape= (432, 7)
    output_shape=(144, 7)
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    #HERE WE START WITH A BIDIRECTIONAL AND WE PUT ALSO CONVO 
    #CONVO CAPTURES SPATIAL CORRELATIONS
    #LSTM CAPTURES BETTER THE TEMPORAL CORRELATIONS
    
    convlstm = tfkl.Bidirectional(tfkl.LSTM(hp.Choice('Bunits', [64,128,256]), return_sequences=True))(input_layer)
    convlstm = tfkl.Conv1D(hp.Choice('Cunits', [128,256,512]), hp.Choice('kernel_size', [3,5,8,12]), padding='same', activation='relu')(convlstm)
    convlstm = tfkl.MaxPool1D()(convlstm)
    convlstm = tfkl.Bidirectional(tfkl.LSTM(hp.Choice('Bunits_2', [128,256,512]), return_sequences=True))(convlstm)
    convlstm = tfkl.Conv1D(hp.Choice('Cunits_2', [256,512,1024]), hp.Choice('kernel_size', [3,5,8,12]), padding='same', activation='relu')(convlstm)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.3)(convlstm)

    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu')(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])

    # Return the model
    return model

In [25]:
#WE NEED TO MANUALLY SPLIT VALIDATION AND TRAINING SET
#we do 0.9 training and 0.1 validation

print(X_train.shape)
split_param= int( X_train.shape[0]*0.9) 
print(split_param)

X_train2 = X_train[: split_param, :, :]
X_val2 = X_train[split_param: , : , :]

print(X_train2.shape)
print(X_val2.shape)

y_train2 = y_train[: split_param, :, :]
y_val2 = y_train[split_param: , : , :]

print(y_train2.shape)
print(y_val2.shape)

In [22]:
#WE INITIALIZE THE TUNER
#we monitor the validation loss obv
#a few iterations just to have an idea
tuner = kt.RandomSearch(
    build_CONV_LSTM_model_for_random_search,
    objective='val_loss',
    max_trials=10)

In [ ]:
#WE DO THE SEARCH
tuner.search(X_train2, y_train2, epochs=10, validation_data=(X_val2, y_val2))
best_model = tuner.get_best_models()[0]

In [ ]:
#WE HAVE A LOOK AT BEST RESULTS
tuner.results_summary()